In [18]:
import datetime  # For datetime objects
import os.path  # To manage paths
from datetime import date, datetime, timedelta

import time
try:
    time_clock = time.process_time
except:
    time_clock = time.clock

# Import the backtrader platform
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats

import quantstats

In [27]:
class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=50), bt.ind.SMA(period=200)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)
        
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') # Comment this line when running optimization
    
    def start(self):
        self.order = None  # sentinel to avoid operrations on pending order

#     def next(self):
#         dt = self.data.datetime.date()

#         portfolio_value = self.broker.get_value()
#         print('%04d - %s - Position Size:     %02d - Value %.2f' %
#               (len(self), dt.isoformat(), self.position.size, portfolio_value))
        
#         size = dt.day
#         percent = size / 100.0

#         print('%04d - %s - Order Target Percent: %.2f' % (len(self), dt.isoformat(), percent))

#         self.order = self.order_target_percent(target=percent)
            
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED: {order.executed.price:.2f}, Size: {order.size}')
            elif order.issell():
                self.log(f'SELL EXECUTED: {order.executed.price:.2f}, Size: {order.size}')
            self.bar_executed = len(self)

        elif order.status == order.Canceled:
            self.log('Order Canceled')

        elif order.status == order.Margin:
            self.log('Order in Margin Call')

        elif order.status == order.Rejected:
            self.log('Order Rejected')

        # Reset orders
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.4f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    def stop(self):
        tused = time_clock() - self.tstart
        if self.p.printdata:
            self.log('Time used: %s' % str(tused))
            self.log('Final portfolio value: %.2f' % self.broker.getvalue())
            self.log('Final cash value: %.2f' % self.broker.getcash())
            self.log('-------------------------')
        else:
            pass

In [28]:
cerebro = bt.Cerebro()

cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.BuySell)

# Set our desired cash start
cerebro.broker.setcash(1000000.0)

# cerebro.addsizer(bt.sizers.AllInSizer, size=100)  # default sizer for strategies

idx = cerebro.addstrategy(SmaCross)
cerebro.addsizer(bt.sizers.AllInSizer)

# cerebro.addstrategy(SmaCross)
# cerebro.addstrategy(MAcrossover)

In [29]:
# Create a Data Feed
PATH = "../data/investpy/data/index/"
data = bt.feeds.GenericCSVData(
    dataname=PATH+'S&P_500.csv',

    fromdate=datetime(2000, 1, 1),
    todate=datetime(2020, 12, 31),

    nullvalue=0.0,

    dtformat=('%Y-%m-%d'),

    datetime=0,
    high=1,
    low=2,
    open=3,
    close=4,
    volume=5,
    openinterest=-1
)

In [30]:
cerebro.adddata(data)

# Add Analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharperatio')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual')
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(btanalyzers.DrawDown)

start_portfolio_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % start_portfolio_value)

results = cerebro.run(maxcpu=4)
strat = results[0]
portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)

end_portfolio_value = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % end_portfolio_value)
pnl = end_portfolio_value - start_portfolio_value
print('PnL: %.2f' % pnl)

# If no name has been specified, the name is the class name lowercased
tret_analyzer = strat.analyzers.getbyname('timereturn')
maxdrawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown']
cagr = strat.analyzers.returns.get_analysis()['rnorm100']
sharpe = strat.analyzers.sharperatio.get_analysis()['sharperatio']
print(f"Max Drawdown: {maxdrawdown:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")
# print('Sharpe Ratio:', strat.analyzers.mysharpe.get_analysis())
# print('Annual Return:', strat.analyzers.myannual.get_analysis())
# print('Drawdown Info:', strat.analyzers.mydrawdown.get_analysis())
# print(strat.get_analysis())


Starting Portfolio Value: 1000000.00
2009-06-24 Order in Margin Call
2010-10-25 Order in Margin Call
2012-02-01 Order in Margin Call
2015-12-22 BUY EXECUTED: 2020.49, Size: 494.76783019568063
2016-01-12 SELL EXECUTED: 1914.35, Size: -494.76783019568063
2016-01-12 OPERATION PROFIT, GROSS -52514.6575, NET -52514.66
2016-04-26 BUY EXECUTED: 2085.80, Size: 453.8221480623197
2018-12-10 SELL EXECUTED: 2583.23, Size: -453.8221480623197
2018-12-10 OPERATION PROFIT, GROSS 225744.7511, NET 225744.75
2019-04-02 BUY EXECUTED: 2858.75, Size: 409.1916104665788
2020-03-31 SELL EXECUTED: 2571.15, Size: -409.1916104665788
2020-03-31 OPERATION PROFIT, GROSS -117683.5072, NET -117683.51
2020-07-10 BUY EXECUTED: 3136.22, Size: 334.87621910930415


AttributeError: 'Lines_LineSeries_LineIterator_DataAccessor_Strateg' object has no attribute 'tstart'

In [ ]:
cerebro.plot()

In [7]:
# quantstats.reports.html(returns, output='./stats/ma_crossover_stats.html', title='Moving Average Crossover')